# Interfacing to Your Own Data

Assumptions:

1. You should already have downloaded a copy or partial copy of ERA5 5.625 degree resolution
2. You have checked out the EDIT monorepo and have a functional EDIT environment into which you can install new packages

This notebook will work through creating a new EDIT package which can interface to the ERA5 dataset, referred to hereafter as "ERA5lowres" for convenience and naming.

This notebook will present two things:

1. The quick install-and-use demo
2. How it was done slowly and carefully so you can do it on new data


## Review of What's on Disk

If you don't already have a replication of the dataset, the zipped data file is 270GB and can be downloaded thusly:

> wget "https://dataserv.ub.tum.de/s/m1524895/download?path=%2F5.625deg&files=all_5.625deg.zip" -O all_5.625deg.zip

The process of unzipping the downloaded file is left as an exercise to the reader. The unzipped data is 471GB. Once downloaded and unpacked, proceed.

In [1]:
# wbench_data_dir = '/g/data/wb00/NCI-Weatherbench/5.625deg'  # on NCI
wbench_data_dir = "/nesi/nobackup/niwa00004/riom/weatherbench/5.625deg/"  # on NIWA HPC
!ls $wbench_data_dir

10m_u_component_of_wind  potential_vorticity	       total_cloud_cover
10m_v_component_of_wind  relative_humidity	       total_precipitation
2m_temperature		 specific_humidity	       u_component_of_wind
constants		 temperature		       v_component_of_wind
geopotential		 temperature_850	       vorticity
geopotential_500	 toa_incident_solar_radiation


In [2]:

!ls $wbench_data_dir/total_cloud_cover/*198*

/nesi/nobackup/niwa00004/riom/weatherbench/5.625deg//total_cloud_cover/total_cloud_cover_1980_5.625deg.nc
/nesi/nobackup/niwa00004/riom/weatherbench/5.625deg//total_cloud_cover/total_cloud_cover_1981_5.625deg.nc
/nesi/nobackup/niwa00004/riom/weatherbench/5.625deg//total_cloud_cover/total_cloud_cover_1982_5.625deg.nc
/nesi/nobackup/niwa00004/riom/weatherbench/5.625deg//total_cloud_cover/total_cloud_cover_1983_5.625deg.nc
/nesi/nobackup/niwa00004/riom/weatherbench/5.625deg//total_cloud_cover/total_cloud_cover_1984_5.625deg.nc
/nesi/nobackup/niwa00004/riom/weatherbench/5.625deg//total_cloud_cover/total_cloud_cover_1985_5.625deg.nc
/nesi/nobackup/niwa00004/riom/weatherbench/5.625deg//total_cloud_cover/total_cloud_cover_1986_5.625deg.nc
/nesi/nobackup/niwa00004/riom/weatherbench/5.625deg//total_cloud_cover/total_cloud_cover_1987_5.625deg.nc
/nesi/nobackup/niwa00004/riom/weatherbench/5.625deg//total_cloud_cover/total_cloud_cover_1988_5.625deg.nc
/nesi/nobackup/niwa00004/riom/weatherbench/5.6

Here we see that the data is stored in subdirectories with human-readable names, and within those subdirectories, the files are names with the year and month of the data. 

This layout is somewhat different to the layout of the full ERA5 dataset as taken directly from CDS, so we will need to tell EDIT how to deal with this difference.

The challenge for EDIT is to understand this directory structure, figure out the shorthand variables names that are actually present in the files (e.g. total_cloud_cover is called "tcc" inside the netcdf file), and work out how to index the whole thing by shorthand-variable-name and date, including interpreting things like the unit of the variable and handling any variable renaming that might be needed for standardisation of naming conventions.

This requires some configuration, and a lot of EDIT's code is about this kind of data set comprehension. 

Let's summarise the easy way.

1. Install the ERA5-lowres Python module
2. Set an environment variable called ERA5LOWRES
3. Import the ERA5-lowres Python module

Assuming you have already installed the ERA5-lowres, let's do step 2 and 3.

In [3]:
%env ERA5LOWRES=$wbench_data_dir

env: ERA5LOWRES=/nesi/nobackup/niwa00004/riom/weatherbench/5.625deg/


In [4]:
import edit.data
import era5lowres

/scale_wlg_persistent/filesets/project/niwa99999/riom/edit_evaluation/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/scale_wlg_persistent/filesets/project/niwa99999/riom/edit_evaluation/EDIT/packages/data/src/edit/data/archive/root.py:24: UserWarning: `ROOT_DIRECTORIES` not found underneath `edit.data.archive`, either archives are not installed or misconfigured. Root Directories cannot be changed. 
  warnings.warn(
/scale_wlg_persistent/filesets/project/niwa99999/riom/edit_evaluation/EDIT/packages/data/src/edit/data/__init__.py:143: FutureWarning: `edit` is under heavy development and may not continue to be supported.
  __python_warnings.warn(


In [5]:
edit.data.archive.era5lowres?

Init signature:
edit.data.archive.era5lowres(
    variables: 'list[str] | str',
    *,
    level_value: 'int | float | list[int | float] | tuple[list | int, ...] | None' = None,
    transforms: 'Transform | TransformCollection | None' = None,
)
Docstring:      ECWMF ReAnalysis v5
Init docstring:
Setup ERA5 Low-Res Indexer

Args:
    variables (list[str] | str):
        Data variables to retrieve
    resolution (Literal[ERA_RES], optional):
        Resolution of data, must be one of 'monthly-averaged','monthly-averaged-by-hour', 'reanalysis'.
        Defaults to 'reanalysis'.
    level_value: (int, optional):
        Level value to select if data contains levels. Defaults to None.
    transforms (Transform | TransformCollection, optional):
        Base Transforms to apply.
        Defaults to TransformCollection().
File:           /scale_wlg_persistent/filesets/project/niwa99999/riom/edit_evaluation/EDIT/packages/era5lowres/src/era5lowres/ERA5DataClass.py
Type:           ABCMeta
Subclas

In [6]:
var=['u', 'v']  # Note - there is no really straightforward way to just list the variables in the archive
                # However, mismatches will cause edit to list what's available with a "did you mean" prompt
                # A specific listing function should be added in future.
UandV = edit.data.archive.era5lowres(var)
UandV

ERA5LowResIndex
	Description                    ECWMF ReAnalysis v5
		 range                          '1970-current'
		 Documentation                  'https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation'


	Initialisation                 
		 level_value                    None
		 variables                      ['u', 'v']
	Transforms                     
		 StandardCoordinateNames        {'latitude': "['lat', 'Latitude', 'yt_ocean', 'yt']", 'longitude': "['lon', 'Longitude', 'xt_ocean', 'xt']", 'replacement_dictionary': 'None', 'time': "['Time']"}
		 Rename                         {'names': {'t2m': "'2t'", 'u10': "'10u'", 'v10': "'10v'", 'siconc': "'ci'"}}

In [7]:
data = UandV['1984-01-01'] 
data

<xarray.Dataset> Size: 5MB
Dimensions:    (latitude: 32, longitude: 64, level: 13, time: 24)
Coordinates:
  * latitude   (latitude) float64 256B -87.19 -81.56 -75.94 ... 81.56 87.19
  * longitude  (longitude) float64 512B 0.0 5.625 11.25 ... 343.1 348.8 354.4
  * level      (level) int32 52B 50 100 150 200 250 300 ... 600 700 850 925 1000
  * time       (time) datetime64[ns] 192B 1984-01-01 ... 1984-01-01T23:00:00
Data variables:
    u          (time, level, latitude, longitude) float32 3MB dask.array<chunksize=(24, 8, 19, 39), meta=np.ndarray>
    v          (time, level, latitude, longitude) float32 3MB dask.array<chunksize=(24, 8, 19, 39), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6

In [8]:
data.u

<xarray.DataArray 'u' (time: 24, level: 13, latitude: 32, longitude: 64)> Size: 3MB
dask.array<getitem, shape=(24, 13, 32, 64), dtype=float32, chunksize=(24, 8, 19, 39), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 256B -87.19 -81.56 -75.94 ... 81.56 87.19
  * longitude  (longitude) float64 512B 0.0 5.625 11.25 ... 343.1 348.8 354.4
  * level      (level) int32 52B 50 100 150 200 250 300 ... 600 700 850 925 1000
  * time       (time) datetime64[ns] 192B 1984-01-01 ... 1984-01-01T23:00:00
Attributes:
    units:          m s**-1
    long_name:      U component of wind
    standard_name:  eastward_wind

In [9]:
data.u.time

<xarray.DataArray 'time' (time: 24)> Size: 192B
array(['1984-01-01T00:00:00.000000000', '1984-01-01T01:00:00.000000000',
       '1984-01-01T02:00:00.000000000', '1984-01-01T03:00:00.000000000',
       '1984-01-01T04:00:00.000000000', '1984-01-01T05:00:00.000000000',
       '1984-01-01T06:00:00.000000000', '1984-01-01T07:00:00.000000000',
       '1984-01-01T08:00:00.000000000', '1984-01-01T09:00:00.000000000',
       '1984-01-01T10:00:00.000000000', '1984-01-01T11:00:00.000000000',
       '1984-01-01T12:00:00.000000000', '1984-01-01T13:00:00.000000000',
       '1984-01-01T14:00:00.000000000', '1984-01-01T15:00:00.000000000',
       '1984-01-01T16:00:00.000000000', '1984-01-01T17:00:00.000000000',
       '1984-01-01T18:00:00.000000000', '1984-01-01T19:00:00.000000000',
       '1984-01-01T20:00:00.000000000', '1984-01-01T21:00:00.000000000',
       '1984-01-01T22:00:00.000000000', '1984-01-01T23:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 192B 1984-01-01 ... 1984-01-01T23:00:00
Attributes:
    long_name:  time